In [ ]:
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import RMSprop ,SGD
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from eXplainableAI.transparent_model.RNN import ConditionalRETAIN

## Load data

In [ ]:
# Not supported data (IRB not permit data sharing to third party )
cwd = '/home/hoheon/Jupyter/2020_01/Noom_RETAIN_PLUS_DTW/'

X_train_var = np.load(cwd + '/cache/numpy/X_train_var.npy')
X_train_inv = np.load(cwd + '/cache/numpy/X_train_fix.npy')
Y_train = np.load(cwd + '/cache/numpy/Y_train.npy')

# test
X_test_var = np.load(cwd + '/cache/numpy/X_test_var.npy')
X_test_inv = np.load(cwd +'/cache/numpy/X_test_fix.npy')
Y_test = np.load(cwd + '/cache/numpy/Y_test.npy')

# Remove weight diff
X_train_var = np.delete(X_train_var, -6, axis=2)
X_test_var = np.delete(X_test_var, -6, axis=2)


vars_ = X_train_var.shape[-1]

print(X_train_var.shape, X_train_inv.shape)

In [ ]:
# Delete N drink, Overcaloires
X_train_var = np.delete(X_train_var, -5, axis=2)
X_train_var = np.delete(X_train_var, -5, axis=2)

X_test_var = np.delete(X_test_var, -5, axis=2)
X_test_var = np.delete(X_test_var, -5, axis=2)

vars_ = X_train_var.shape[-1]

print(X_train_var.shape, X_train_inv.shape)

## Main

In [ ]:
config = {'n_features':vars_,
          'n_auxs':9,
          'steps':16,
          'hidden_units': 20
          }

conditional_retain = ConditionalRETAIN(config)
retain = conditional_retain.build_model()
retain.compile(optimizer=RMSprop(lr=0.0005), loss='mse', metrics=['mse'])
retain.summary()

In [ ]:
model_dir = './model/'
check_point = ModelCheckpoint(filepath=model_dir+'{epoch:03d}.hdf5',
                     monitor='val_loss', 
                     verbose=1, 
                     save_best_only=True, 
                     save_weights_only=True,
                     mode='min' 
                     )

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001, mode='min')  

history = retain.fit(x=[X_train_var, X_train_inv], 
                     y=Y_train,   
                     batch_size=16*5,
                     epochs=500,
                     shuffle=False,
                     callbacks=[check_point, reduce_lr],
                     validation_split=0.3)